In [3]:
import h5pyd
import requests
import pandas as pd

### DATA PROCESSING STEPS
##### 1. acquiring API KEY
##### 2. specifying parameters of interest
##### 3. wrangling the api response; reading it as a text and string manipulation to read it into a df
##### 4. save pandas df

In [ ]:
LATLONG = "-87.669888 42.052294"
API_KEY = "19dv992vs4tNDqBmz2qX5UIWERNrFtyNcHoX6JdH"

base_url = "https://developer.nrel.gov/api/wind-toolkit/v2/wind/offshore-great-lakes-download.csv?"
params = f"wkt=POINT({LATLONG})&attributes=wind_speed,wind_direction,pressure,temperature&names=2012&utc=false&leap_day=true&full_name=Mark%20Roth&email=rothmark%40oregonstate.edu&api_key={API_KEY}"
z = requests.get(base_url+params)

In [ ]:
lines = z.text.split("\n")
meta_data = lines[0:2]
header_names = lines[2:3]

In [ ]:
rows = [x.split(",") for x in lines[3:]]
df = pd.DataFrame(rows)
print('done')

In [ ]:
df.columns = header_names[0].split(",")
df.head()

In [ ]:
df.to_csv("~/Desktop/NREL/lakefill_df.csv", index=False)

### Data Exploration

#### Assumptions:
- diurnal means occurring in the daytime (e.g., looking at 6am-6pm of each day) as opposed to daily
- I made this assumption because energy consumption varies drastically during the day vs the night
- we can change the interpretation of diurnal to mean daily if we switch the "IS_DAILY" flag to True

In [165]:
import plotly.express as xp
import matplotlib
import plotly.figure_factory as ff
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import date
from statsmodels.tsa.stattools import pacf, acf
from copy import copy

In [ ]:
df = pd.read_csv("~/Documents/NREL/lakefill_df.csv")
# if not IS_DAILY:
#     df = df[df['Hour'] < 18]
#     df = df[df['Hour'] > 5]
# diurnal_df = df.groupby(['Month', 'Day'], as_index=False).mean()
# df_std = df[['Month', 'Day', 'wind speed at 100m (m/s)']].groupby(['Month', 'Day'], as_index=False).agg(np.std)
# diurnal_df['wind speed std'] = df_std['wind speed at 100m (m/s)']
# diurnal_df = df.groupby(['Month', 'Day'], as_index=False).agg(['mean', np.std])

In [ ]:
# df.columns
# z = df.drop("Hour", axis=1)


In [198]:
# can be made to take a continuous variable (i.e., number of days to aggregate) 
# rather than a categorical variable
def filter_df_by_time(df, scale, rolling_unit=7, is_daily=True):
    """
    function to filter a dataframe to examine different temporal scales

    :param df:      dataframe for filtering; should have the following
                    columns: Hour, Day, Month, Year, wind speed at
                    100m (m/s)

    :param scale:   desired temporal scale for analysis; should be one
                    of the following: MONTHLY, HOURLY, DIURNAL

    :param rolling_unit:   unit of measurement to calculate rolling avg

    :return:        df filtered by the way specified in scale, with the
                    addition of (1) the wind speed std calc over the
                    same scale and (2) unit rolling average
    """

    if scale == DIURNAL:
        if not is_daily:
            df = df[df['Hour'] < 19]
            df = df[df['Hour'] > 5]
        groupby_cols = ['Month', 'Day']
        gb_speed = ['Month', 'Day', 'wind speed at 100m (m/s)']
    
    elif scale == MONTHLY:
        groupby_cols = ['Month']
        gb_speed = ['Month', 'wind speed at 100m (m/s)']

    elif scale == HOURLY:
        groupby_cols = ['Month', 'Day', 'Hour']
        gb_speed = ['Month', 'Day', 'Hour', 'wind speed at 100m (m/s)']

    df_std = df[gb_speed].groupby(groupby_cols, as_index=False).agg(np.std)
    df = df.groupby(groupby_cols, as_index=False).mean()
    df['wind speed std'] = df_std['wind speed at 100m (m/s)']
    df['rolling avg'] = df['wind speed at 100m (m/s)'].rolling(rolling_unit).mean()
    df['date'] = pd.to_datetime(
        df[['Month', 'Day', 'Year']],
        infer_datetime_format=True
    )
    df['date+hour'] = pd.to_datetime(
        df[['Hour', 'Month', 'Day', 'Year']],
        infer_datetime_format=True
    )

    # print("filter by time: " + df.columns)
    return df


def base_graph(df, scale, highlight_type, var_of_interest="wind speed at 100m (m/s)", thresh=7):
    x_axis = 'date'
    if scale == HOURLY:
        x_axis = 'date+hour'

    # TODO: mix btwn hard code and variable var of interest
    t0 = go.Bar(
        x=df[x_axis],
        y=df[var_of_interest],
        error_y=dict(
                type='data',
                array=df["wind speed std"],
                visible=True),
        name="wind speed at 100m (m/s)"
    )
    t1 = go.Scatter(x=df[x_axis], y=[df[var_of_interest].mean()]*len(df), name="avg wind speed")
    t2 = go.Scatter(x=df[x_axis], y=df["rolling avg"], name=f"{thresh} unit average: {scale}")

    if highlight_type in ["top10", "above avg", "rolling avg"]:
        # this categorical variable could easily be turned into a slider (continuous)
        if highlight_type in ["top10", "above avg"]:
            if highlight_type == "top10":
                q = .9
                q_val = df[var_of_interest].quantile(q)
                perc = 10
            elif highlight_type == "above avg":
                q_val = df[var_of_interest].mean()
                perc = 50

            hi_df = df[df[var_of_interest] >= q_val]
            reg_df = df[df[var_of_interest] < q_val]
            tr_name = f"TOP {perc}% of wind speeds"


        # find idx with highest 7 day average and work 7 idx back from that
        if highlight_type == "rolling avg":
            idx = df[var_of_interest].idxmax()
            hi_df = df.iloc[(idx):(idx+thresh)]
            reg_df = df.drop([x for x in range(idx, (idx+thresh))])
            tr_name = f"Greatest 7 Unit Sum of Wind Speed: {scale}"
    
        t0 = go.Bar(
            x=reg_df[x_axis],
            y=reg_df[var_of_interest],
            error_y=dict(
                    type='data',
                    array=reg_df["wind speed std"],
                    visible=True),
            name="wind speed at 100m (m/s)"
        )

        t3 = go.Bar(
            x=hi_df[x_axis],
            y=hi_df[var_of_interest],
            error_y=dict(
                    type='data',
                    array=hi_df["wind speed std"],
                    visible=True),
            name=tr_name,
            marker_color="yellow"
        )

    traces = [t0, t1]
    
    if scale in [DIURNAL, HOURLY]:
        traces.append(t2)

    if highlight_type is not None:
        traces.append(t3)

    fig = go.Figure(data=traces)
    fig.update_layout(
        title=f"{scale} Wind Speed Variability",
        xaxis_title="Date",
        yaxis_title="Wind Speed at 100m (m/s)"
    )
    return fig


def agg_by(df, scale="Hour", var_of_interest="wind speed at 100m (m/s)"):
    if not IS_DAILY:
        df = df[df['Hour'] < 19]
        df = df[df['Hour'] > 5]
    df = df.groupby([scale], as_index=False).mean()
    return df[[scale, var_of_interest]]


def heat_map_corr_mat(df, var_of_interest='wind speed at 100m (m/s)'):
    EXCLUDE = ["Hour", "Day", "date", "date+hour", "Year", "Month", "Minute", "rolling avg", "wind speed std"]
    to_keep = list(set(df.columns) - set(EXCLUDE))
    df_corr = df[to_keep]
    cors = []
    for x in df_corr.columns:
        cors.append(str(round(df_corr[var_of_interest].corr(df_corr[x]), 4)))

    fig = go.Figure(data=[go.Table(
        header=dict(values=['Variable', 'Correlation'],
                    line_color='darkslategray',
                    fill_color='lightgreen',
                    align='center'),
        cells=dict(values=[df_corr.columns, # 1st column
                           cors], # 2nd column
                   line_color='darkslategray',
                   fill_color='lightcyan',
                   align='center'))
    ])
    fig.update_layout(title="Correlation Table")
    return fig


def plot_autocorrelation(df, is_partial, var_of_interest="wind speed at 100m (m/s)"):
    if is_partial:
        df_pacf = pacf(df[var_of_interest])
    else:
        df_pacf = acf(df[var_of_interest])
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x= np.arange(len(df_pacf)),
        y= df_pacf,
        name= 'PACF',
    ))
    fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(
        title="Partial Autocorrelation",
        xaxis_title="Lag",
        yaxis_title="Partial Autocorrelation",
    )
    return fig


def show_percentile(df, percentiles = None , var_of_interest='wind speed at 100m (m/s)'):
    if percentiles is None:
	    percentiles = [.1, .25, .5, .75, .9]
    fig = go.Figure(data=[go.Table(
    header=dict(values=['Percentile', 'Wind Speed (m/s)'],
                line_color='darkslategray',
                fill_color='lightgreen',
                align='center'),
    cells=dict(values=[[x*100 for x in percentiles], # 1st column
                       [round(df[var_of_interest].quantile(perc), 3) for perc in percentiles]], # 2nd column
               line_color='darkslategray',
               fill_color='lightcyan',
               align='center'))
    ])
    fig.update_layout(title="Percentiles Table")
    return fig


def calc_most_consec_days(di_df, thresh='mean', var_of_interest='wind speed at 100m (m/s)'):
    temp1 = 'Month'
    temp2 = 'Day'

    di_df['most_consec_days'] = -1
    if thresh == 'mean':
        avg_val = di_df[var_of_interest].mean()
    else:
        avg_val = thresh

    for m in set(di_df[temp1]):
        df_m = di_df[di_df[temp1] == m]
        count = 0
        max_ct = 0
        for d in set(df_m[temp2]):
            df_m_d = df_m[df_m[temp2] == d]
            # print(df_m_d[var_of_interest].values[0])
            if df_m_d[var_of_interest].values[0] >= avg_val:
                count += 1
                if count > max_ct:
                    max_ct = count
            else:
                count = 0
        di_df.most_consec_days[(di_df[temp1] == m)] = max_ct

    return di_df[['Year', 'Day', 'Month', 'most_consec_days']].groupby('Month', as_index=False).mean()
    # return df[['date', 'Day', 'Month', 'most_consec_days']].groupby('Day', 'Month').mean()

def calc_most_consec_hours(df_hours, thresh='mean', var_of_interest='wind speed at 100m (m/s)'):

    temp1 = 'Month'
    temp2 = 'Day'
    temp3 = 'Hour'

    if thresh == 'mean':
        avg_val = df_hours[var_of_interest].mean()
    else:
        avg_val = thresh
    df_hours['most_consec_hours'] = -1
    for m in set(df_hours[temp1]):
        df_m = df_hours[df_hours[temp1] == m]
        for d in set(df_m[temp2]):
            df_m_d = df_m[df_m[temp2] == d]
            count = 0
            max_ct = 0
            for h in set(df_m_d[temp3]):
                df_m_d_h = df_m_d[df_m_d[temp3] == h]
                # print(df_m_d[var_of_interest].values[0])
                if df_m_d_h[var_of_interest].values[0] >= avg_val:
                    count += 1
                    if count > max_ct:
                        max_ct = count
                else:
                    count = 0
            df_hours.most_consec_hours[(df_hours[temp1] == m) & (df_hours[temp2] == d)] = max_ct

    return df_hours[['Year', 'Day', 'Month', 'most_consec_hours']].groupby(['Month', 'Day'], as_index=False).mean()


def show_consec_graphs(df_to_plot, scale, var_of_interest='wind speed at 100m (m/s)'):
    x_axis = 'date'
    if scale == DIURNAL:
        # df_to_plot = CONSEC_HOURS
        var_of_interest = "most_consec_hours"
        title_name = "Hours"
    elif scale == MONTHLY:
        # df_to_plot = CONSEC_DAYS
        var_of_interest = "most_consec_days"
        title_name = "Days"

    t0 = go.Bar(
        x=df_to_plot[x_axis],
        y=df_to_plot[var_of_interest],
        name=f"Consecutive High Wind Speeds {title_name}",
    )

    fig = go.Figure(data=[t0])
    fig.update_layout(
        title=f"Most Consecutive {title_name} of High Wind Speed Per Time Unit",
        xaxis_title="Date",
        yaxis_title=f"Consecutive {title_name}"
    )
    return fig

# def show_consec_stats(df, scale, thresh):
#     temp1 = 'Month'
#     temp2 = 'Day'
#     if scale == DIURNAL:
#         temp3 = 'Hour'
#     # elif scale == MONTHLY:
#         # temp1 = 'Day'
#
#     for m in df[[temp1]]:
#         for d in df[[temp2]]:




## DASH components setup

In [199]:
import dash
from dash import dcc
from dash import html
from dash import Input, Output

In [ ]:
DIURNAL = "Diurnal"
MONTHLY = "Monthly"
HOURLY = "Hourly"
IS_DAILY = True
# sacle: replace this w a curl request & data filtering from above
df = pd.read_csv("~/Documents/NREL/lakefill_df.csv")
df['date'] = pd.to_datetime(
    df[['Month', 'Day', 'Year']],
    infer_datetime_format=True
)
df['date+hour'] = pd.to_datetime(
    df[['Hour', 'Month', 'Day', 'Year']],
    infer_datetime_format=True
)
print("starting to calc consec hours...")

CONSEC_HOURS = calc_most_consec_hours(copy(df))
print("done precessing hours...")
DI_DF = filter_df_by_time(df, DIURNAL, rolling_unit=7, is_daily=IS_DAILY)
print("starting to calc consec days...")
CONSEC_DAYS = calc_most_consec_days(copy(DI_DF))
print("done precessing days...")
MN_DF = filter_df_by_time(df, MONTHLY, rolling_unit=2)

CONSEC_DAYS['date'] = pd.to_datetime(
        CONSEC_DAYS[['Month', 'Day', 'Year']],
        infer_datetime_format=True
)

CONSEC_HOURS['date'] = pd.to_datetime(
    CONSEC_HOURS[['Month', 'Day', 'Year']],
    infer_datetime_format=True
)

app = dash.Dash()
app.layout = html.Div([
    dcc.Dropdown(
        id='base-graph-dropdown',
        options=[
            {'label': 'Diurnal', 'value': DIURNAL},
            {'label': 'Monthly', 'value': MONTHLY},
        ],
        value=DIURNAL
    ),
    dcc.Dropdown(id='highlight-graph-dropdown'),
    dcc.DatePickerRange(
        id='date-range-base',
        min_date_allowed=date(2012, 1, 1),
        max_date_allowed=date(2012, 12, 31),
        initial_visible_month=date(2012, 1, 1),
        start_date=date(2012, 1, 1),
        end_date=date(2012, 12, 31)
    ),
    html.Div(id='base-graph-output-container'),

    dcc.Dropdown(
        id='bar-x-axis',
        options=[
            {'label': 'Avg Wind by Hour', 'value': 'aWbH'},
            {'label': 'Correlation Matrix', 'value': 'CorMat'},
            {'label': 'Plot Autocorrelation', 'value': 'ACorr'},
            {'label': 'Plot Partial Autocorrelation', 'value': 'PACorr'},
            {'label': 'Show Wind Speed Percentiles', 'value': 'percentile'},
            # vv todo make this dynamic based on temporal scale to only show one of these vv
            {'label': 'Most Number of Consecutive High Wind Hours (DIURNAL)', 'value': 'hi_diurnal'},
            {'label': 'Most Number of Consecutive High Wind Days (MONTHLY', 'value': 'hi_monthly'},
        ],
        value="aWbH"
    ),
    html.Div(id='bar-output-container')
])

@app.callback(
    Output('highlight-graph-dropdown', 'options'),
    Input('base-graph-dropdown', 'value'),
)
def update_dropdown(temporal_scale):
    if temporal_scale in [DIURNAL, HOURLY]:
        return [
            {'label': 'Top 10%', 'value': 'top10'},
            {'label': 'Above Average', 'value': 'above avg'},
            {'label': f'Largest 7 {temporal_scale} Period', 'value': 'rolling avg'},
        ]
    elif temporal_scale == MONTHLY:
        return [
            {'label': 'Top 10%', 'value': 'top10'},
            {'label': 'Above Average', 'value': 'above avg'},
        ]


@app.callback(
    Output('base-graph-output-container', 'children'),
    Input('base-graph-dropdown', 'value'),
    Input('highlight-graph-dropdown', 'value'),
    Input('date-range-base', 'start_date'),
    Input('date-range-base', 'end_date')
)
def update_output(temp_scale, highlight_type, start_date, end_date):

    if temp_scale == DIURNAL:
        df_to_plot = DI_DF
    elif temp_scale == MONTHLY:
        df_to_plot = MN_DF

    # elif temp_scale == HOURLY:
    #     df_to_plot = filter_df_by_time(df, HOURLY, rolling_unit=5)
    df = df_to_plot[(df_to_plot['date'] >= start_date) & (df_to_plot['date'] <= end_date)]
    fig = base_graph(df, temp_scale, highlight_type)
    return dcc.Graph(figure=fig)


@app.callback(
    Output('bar-output-container', 'children'),
    Input('bar-x-axis', 'value'),
    Input('base-graph-dropdown', 'value'),
    Input('date-range-base', 'start_date'),
    Input('date-range-base', 'end_date')
)
# TODO: how to pass objects between functions? does the app have state?
def update_secondary(bar_type, temp_input, start_date, end_date, df_base=df):

    if bar_type == "aWbH":
        # it would be nice to have this change based on the
        # dynamic x-range that plotly offers
        df = df_base[(df_base['date'] >= start_date) & (df_base['date'] <= end_date)]

        hist_df = agg_by(df)
        t0 = go.Bar(
            x=hist_df["Hour"],
            y=hist_df["wind speed at 100m (m/s)"],
            name="wind speed at 100m (m/s)"
        )
        traces = [t0]
        fig = go.Figure(data=traces)
        fig.update_layout(title=f"Wind Speed Variability X Hour")

    elif bar_type in ["hi_diurnal", "hi_monthly"]:
        if temp_input == DIURNAL:
            df_to_plot = CONSEC_HOURS
        elif temp_input == MONTHLY:
            df_to_plot = CONSEC_DAYS

        df_to_plot = df_to_plot[(df_to_plot['date'] >= start_date) & (df_to_plot['date'] <= end_date)]
        fig = show_consec_graphs(df_to_plot, temp_input)

    else:
        if temp_input == DIURNAL:
            df_to_plot = DI_DF
        elif temp_input == MONTHLY:
            df_to_plot = MN_DF
        # elif temp_input == HOURLY:
        #     df_to_plot = filter_df_by_time(df, HOURLY, rolling_unit=5)
        else:
            exit(f"ERROR; bar_type not {DIURNAL} nor {MONTHLY}")

        df_to_plot = df_to_plot[(df_to_plot['date'] >= start_date) & (df_to_plot['date'] <= end_date)]

        if bar_type == "CorMat":
            fig = heat_map_corr_mat(df_to_plot)
        elif bar_type == "PACorr":
            fig = plot_autocorrelation(df_to_plot, is_partial=True)
        elif bar_type == "ACorr":
            fig = plot_autocorrelation(df_to_plot, is_partial=False)
        elif bar_type == "percentile":
            fig = show_percentile(df_to_plot)

    return dcc.Graph(figure=fig)

app.run_server(debug=True, use_reloader=False)

starting to calc consec hours...


In [177]:
def prep_data():
    df = pd.read_csv("~/Documents/NREL/lakefill_df.csv")
    df['date'] = pd.to_datetime(
        df[['Month', 'Day', 'Year']],
        infer_datetime_format=True
    )
    df['date+hour'] = pd.to_datetime(
        df[['Hour', 'Month', 'Day', 'Year']],
        infer_datetime_format=True
    )
    print("starting to calc consec hours...")

    CONSEC_HOURS = calc_most_consec_hours(copy(df))
    print("done precessing hours...")
    DI_DF = filter_df_by_time(df, DIURNAL, rolling_unit=7, is_daily=IS_DAILY)
    print("starting to calc consec days...")
    CONSEC_DAYS = calc_most_consec_days(copy(DI_DF))
    print("done precessing days...")
    MN_DF = filter_df_by_time(df, MONTHLY, rolling_unit=2)

    CONSEC_DAYS['date'] = pd.to_datetime(
            CONSEC_DAYS[['Month', 'Day', 'Year']],
            infer_datetime_format=True
    )

    CONSEC_HOURS['date'] = pd.to_datetime(
        CONSEC_HOURS[['Month', 'Day', 'Year']],
        infer_datetime_format=True
    )

     Year  Month  Day  Hour  Minute  wind speed at 100m (m/s)  \
0  2012.0    1.0  1.0   0.0    30.0                     14.86   
1  2012.0    1.0  1.0   1.0    30.0                     14.76   
2  2012.0    1.0  1.0   2.0    30.0                     15.26   
3  2012.0    1.0  1.0   3.0    30.0                     14.56   
4  2012.0    1.0  1.0   4.0    30.0                     13.45   

   wind direction at 100m (deg)  air pressure at 100m (Pa)  \
0                        179.16                    97110.0   
1                        186.39                    96950.0   
2                        192.45                    96820.0   
3                        203.30                    96680.0   
4                        217.14                    96530.0   

   air temperature at 100m (C)       date           date+hour  
0                         4.57 2012-01-01 2012-01-01 00:00:00  
1                         4.94 2012-01-01 2012-01-01 01:00:00  
2                         5.11 2012-01-01 20